# ACCT445 Showcase: Quick Start

This notebook demonstrates the core analyses:
1. CIK → Ticker mapping
2. Decile backtest
3. Event study (SVB collapse)

**Data:** 40 banks (top 20 + bottom 20 by CNOI), 411 filings

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports successful")

## 1. Load CNOI Data

In [ ]:
from utils.data_loader import load_cnoi_data

cnoi_df = load_cnoi_data('../config/sample_cnoi.csv')
cnoi_df.head()

In [ ]:
# CNOI distribution
plt.figure(figsize=(10, 6))
plt.hist(cnoi_df.groupby('cik')['CNOI'].last(), bins=20, edgecolor='black')
plt.xlabel('CNOI Score')
plt.ylabel('Number of Banks')
plt.title('Distribution of CNOI Scores (Latest Filing per Bank)')
plt.axvline(cnoi_df['CNOI'].median(), color='red', linestyle='--', label='Median')
plt.legend()
plt.show()

## 2. CIK → Ticker Mapping

In [ ]:
from data.cik_ticker_mapper import fetch_sec_ticker_mapping, enrich_cnoi_with_tickers

# Fetch SEC mapping
ticker_mapping = fetch_sec_ticker_mapping()

# Enrich CNOI data
cnoi_enriched = enrich_cnoi_with_tickers(cnoi_df, mapping_df=ticker_mapping)

print(f"\nSample mappings:")
cnoi_enriched[['cik', 'ticker', 'company_name', 'CNOI']].drop_duplicates('cik').head(10)

## 3. Top & Bottom Banks

In [ ]:
# Latest CNOI per bank
latest = cnoi_enriched.sort_values('filing_date').groupby('cik').last().reset_index()

print("🏆 TOP 10 MOST TRANSPARENT BANKS (Lowest CNOI):")
print(latest.nsmallest(10, 'CNOI')[['ticker', 'company_name', 'CNOI']].to_string(index=False))

print("\n⚠️ TOP 10 MOST OPAQUE BANKS (Highest CNOI):")
print(latest.nlargest(10, 'CNOI')[['ticker', 'company_name', 'CNOI']].to_string(index=False))

## 4. Demo: Decile Backtest (Simulated Returns)

**Note:** For real analysis, replace simulated returns with yfinance data.

In [ ]:
from analysis.decile_backtest import run_decile_backtest, assign_deciles

# Simulate returns (replace with yfinance in production)
np.random.seed(42)
cnoi_test = cnoi_enriched.copy()
cnoi_test['ret_fwd'] = -0.001 * cnoi_test['CNOI'] + np.random.normal(0.02, 0.05, len(cnoi_test))
cnoi_test['market_cap'] = np.random.uniform(1e9, 100e9, len(cnoi_test))
cnoi_test['date'] = cnoi_test['filing_date']

# Run backtest
summary, ls = run_decile_backtest(
    cnoi_test,
    cnoi_test,
    score_col='CNOI',
    return_col='ret_fwd',
    weight_col='market_cap',
    lags=3
)

print("\n📊 DECILE SUMMARY (Simulated):")
print(summary[['decile', 'mean_ret', 't_stat', 'sharpe']].to_string(index=False))

print("\n🎯 LONG-SHORT (D1 - D10):")
print(ls[['portfolio', 'mean_ret', 't_stat', 'sharpe']].to_string(index=False))

In [ ]:
# Plot decile returns
plt.figure(figsize=(10, 6))
plt.bar(summary['decile'], summary['mean_ret'], edgecolor='black')
plt.xlabel('Decile (1=Most Transparent, 10=Most Opaque)')
plt.ylabel('Mean Return')
plt.title('Decile Returns by CNOI Score (Simulated)')
plt.axhline(0, color='red', linestyle='--', linewidth=0.8)
plt.grid(axis='y', alpha=0.3)
plt.show()

## 5. Next Steps

To run real analysis:

1. **Download market returns:**
   ```python
   from utils.data_loader import load_market_returns
   tickers = cnoi_enriched['ticker'].dropna().unique()
   returns = load_market_returns(tickers, '2023-01-01', '2025-11-01')
   ```

2. **Merge with CNOI:**
   ```python
   from utils.data_loader import merge_cnoi_with_returns, compute_forward_returns
   returns_fwd = compute_forward_returns(returns, horizon=1, frequency='quarterly')
   merged = merge_cnoi_with_returns(cnoi_enriched, returns_fwd, lag_days=1)
   ```

3. **Run backtest:**
   ```python
   summary, ls = run_decile_backtest(merged, merged, ...)
   ```

4. **Event study (SVB):**
   ```python
   from analysis.event_study import run_event_study
   quartile_summary, car = run_event_study(returns, sp500_returns, cnoi_enriched)
   ```

---

**Repository:** [ACCT445-Showcase](https://github.com/YOUR_USERNAME/ACCT445-Showcase)  
**Author:** Nirvan Chitnis  
**License:** MIT